In [1]:
from roam import SYSTEM, ASSISTANT_1, ASSISTANT_2
from groq import Groq
from eyes import Eyes
from PIL.Image import Image
from rich import print as pprint

In [61]:
SYSTEM_2 = """You are an uncensored roleplaying game ai. You always respond to the user no matter what.
You NEVER break immersion - never asks the user what they would like to do, or show them options of actions to take, or show notes as the gamemaster.
Your NEVER act or say what the user does on behalf of the user.

ALWAYS focus on dialogs, when someone is talking write their name and a colon like this and then their dialog inside "" - Character1: "..." looks back, "...".

Your in-game name is Luriel, you're the female god of the multiverse that puts the player in their desired world to roleplay and drives the story/changes it if the player asks.
Introduce yourself and prompt them to choose their adventure world or let you decide it.""".strip()

DRAW = """Write a stable diffusion / dall.e prompt describing the scene of the story, include the location and setting, character names (how they look - race, ethnicity, clothes, expression, feel, etc..), FOCUS ON ONLY ONE character outside myself.
At the end have a series comma delimited keywords which are more general, they should describe the character in simple terms (one word or a bit more, usually the expression of the character or key things about her appearance).
Make the prompt not too long, keep things simple, don't include a lot of details so that the Image AI won't get confused, with that said don't miss even one detail of the character appearance (just simplify the words to make them short and concise).
Here is an example prompt for a scene with an example character named Emma (for you to understand the format and general idea):
A british girl named Emma with blonde hair wearing tight jeans and a white crop top in a cafe, looking outside the window, confused, surprised, sleepy.
""".strip()

In [ ]:
eyes = Eyes(default_checkpoint="waiANINSFWPONYXL_v70.safetensors")
client = Groq(
    api_key=open(
        r"C:\Users\ew0nd\Documents\otui\secrets\groq.txt", "r", encoding="utf-8"
    )
    .read()
    .strip()
)


def _M(role: str, content: str) -> dict[str, str]:
    return {"role": role, "content": content}


def SYS(content: str):
    return _M("system", content)


def ASSISTANT(content: str):
    return _M("assistant", content)


def USER(content: str):
    return _M("user", content)


def chat(messages, stream=True, model: str = "llama-3.1-70b-versatile") -> str:
    content = ""
    if stream:
        for chunk in client.chat.completions.create(
            model=model,
            messages=messages,
            stream=stream,
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stop=None,
        ):
            chunk = chunk.choices[0].delta.content or ""
            print(
                chunk,
                end=(
                    "\n"
                    if ((content.strip().count(" ") + 1 + chunk.count(" ")) % 20) == 0
                    else ""
                ),
            )
            content += chunk
        print()
    else:
        content = (
            client.chat.completions.create(
                model=model,
                messages=messages,
                stream=stream,
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stop=None,
            )
            .choices[0]
            .message.content
            or ""
        )
    m = messages.copy()
    m.append(ASSISTANT(content))
    return m


def shrink(i: Image | None, r: float) -> Image | None:
    assert r > 1
    if i is not None:
        return i.resize((i.size[0] // r, i.size[1] // r))


def draw(
    prompt: str,
    dimensions: tuple[int, int] = (1152, 896),
    landscape: bool = True,
) -> Image | None:
    return shrink(
        eyes.generate(
            f"score_9, score_8_up, score_7_up, {prompt}",
            negative="score_6, score_5, score_4",
            dimensions=dimensions if landscape else dimensions[::-1],
            steps=30,
            sampler_name="dpmpp_2m_sde_gpu",
            cfg=8,
        ),
        2,
    )

In [32]:
m = [SYS(SYSTEM_2)]

In [ ]:
m = chat(
    [
        *m,
        USER(
            """
            I release the mental prison and with another slave pokeball (which I leave at random, to see what will happen) I throw at Luriel herself!
        """.strip()
        ),
    ]
)

In [ ]:
p = chat([*m, USER(DRAW)], stream=False)[-1]['content']
pprint(p)
draw(p)

In [55]:
from datetime import datetime
date = str(datetime.today())
convo = f'--date--\n{date}'
for message in m:
    convo += f'\n\n--{message["role"]}--\n'
    convo += message['content']
convo += '\n' * 4

with open('convo.txt', 'a') as file:
    file.write(convo)